In [43]:
import re
import requests
from bs4 import BeautifulSoup
import time
from urllib import parse
import json
from selenium import webdriver


#@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@
# location_name_list 뽑는 과정 

def location_name_list_crawler():
    request = requests.get("http://www.yapen.co.kr")
    response = request.text
    soup = BeautifulSoup(response, 'lxml')

    left_menu=soup.select('div.locLayer')
    # 풀빌라, MD추천 제외 14지역중 7지역 만남김.
    selected_left_menu= left_menu[2:9]
    

    # 여기에 list 형태로 지역,지역고유번호/(세부지역,고유번호) 넣고싶다. 
    # location_name_list =[ [지역1 ,지역1고유번호 , [ (고유번호,세부지역),(고유번호2,세부지역2)...] ],
    #                       [지역2 ,지역2고유번호 , [ (고유번호,세부지역),(고유번호2,세부지역2)...] ],..
    location_name_list=list()

    for location in selected_left_menu:
        # 지역 이름 먼저 뽑음 
        location_name = location.select('div.titleStyle')[0].get_text(strip=True)
        location_name_sub_list=list()
        location_name_sub_list.append(location_name)

        li=location.select('li')

        #for문 돌면서 (고유번호 세부지역) 리스트에 담은뒤 location_list에 넣겠다.
        location_detail_list=[] 
        for location_detail in li:
            onclick_value = location_detail['onclick']  #regionMove('1.003021','금산/논산');
            
            split_right = onclick_value.split(',')[0]
            split_left = onclick_value.split(',')[1]

            sub_location_no = re.findall("'(.+)'",split_right)[0]
            sub_location_name = re.findall("'(.+)'",split_left)[0]

            location_detail_tuple = (sub_location_no,sub_location_name)

             # (고유번호/세부지역) 리스트에 담음
            location_detail_list.append(location_detail_tuple)
        
        # 지역 고유번호부터 정규표현식으로 뽑아내서 담음.(세부지역 고유번호의 소숫점 뒤 3자리)
        
        location_detail_no_for_search = location_detail_list[0][0] # '1.003021'
        location_detaol_no = re.findall(".(\d\d\d)",location_detail_no_for_search)[0]
        location_name_sub_list.append(location_detaol_no)
        
        # location_name_sub_list (고유번호 세부지역) 리스트를 넣음.
        location_name_sub_list.append(location_detail_list)
        
        # 상위 리스트에 넣음 
        location_name_list.append(location_name_sub_list)
        
    return location_name_list
    
    
# location_dict 뽑는 과정 끝
######################################################################################












#@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@
#이제 각 페이지 location 가서 해커톤때 쓴 pension crawler 돌리고 싶다.
# 메인페이지에서 기본정보 3개만 여러개 팬션에게서 가져왔던것.

# url = "http://www.yapen.co.kr/region?location=015&subLocation=1.015001"

def pension_crawler(location_no,sub_location_no):

    params = {
            'location':location_no,
             'subLocation':sub_location_no,
        }

    url = "http://www.yapen.co.kr/region?" + parse.urlencode(params)

    request = requests.get(url)
    response = request.text
    soup = BeautifulSoup(response, 'lxml')


    title_list=list()
    img_file_list=list()
    price_list=list()
    pldx_list=list()
    discount_rate_list=list()

    title_uls = soup.select('ul.dest-place-opt-fea')
    for ul in title_uls:
        li=ul.select('li')
        title_list.append(li[1].get_text())

    price_uls = soup.select('ul.dest-place-opt-cast')
    for ul in price_uls:
        li=ul.select('li')
        price_list.append(li[1].get_text())


    img_file_divs = soup.select('div.imgBox')
    for div in img_file_divs:
        img_file_list.append(div.select('img')[0]['src'])

        list1 = re.split('/', div.select('img')[0]['src'])
        pldx_list.append(int(list1[5]))


    dest_place_pics = soup.select('div.dest-place-pic')
    for dest_place_pic in dest_place_pics:
        # dest_place_pic에는 dic가 2개 or 1게 있는데  discount_rate가 있는 경우는 div가 2개이며
        # 길이가 5여서 이것으로 discount_rate있고 없고 를 비교한다. 
        if(len(dest_place_pic)==5):
            discount_rate_string =dest_place_pic.select('div')[0].get_text(strip=True)
            # %문자 정규표현식으로 빼줌.
            discount_rate_int = int(re.search('(\d*)',discount_rate_string).group())
            discount_rate_list.append(discount_rate_int)
        else:
            discount_rate_list.append(0)


    sub_locations_info_list = [title_list, price_list, img_file_list,pldx_list,discount_rate_list]
    
    return sub_locations_info_list
        

###########################################################################################







###########################################################################################
##location_name_list 로부터 지역명,고유번호 받아서 각 세부지역별로 pension_crawler 
## 해서 기본정보 5개씩 모으는 크롤러##

def location_crawler():
    
    location_info_list=list()
    
    location_name_list = location_name_list_crawler()
    # location_name_list =[ [지역1 ,지역1고유번호 , [ (고유번호,세부지역),(고유번호2,세부지역2)...] ],
    #                       [지역2 ,지역2고유번호 , [ (고유번호,세부지역),(고유번호2,세부지역2)...] ],..
    
    for location in location_name_list:
        location_name = location[0]
        location_no = location[1]
        sub_location_list = location[2]
        for sub_location in sub_location_list:
            print(sub_location)
            sub_location_no = sub_location[0]
            sub_location_name = sub_location[1]
            sub_locations_info_list = pension_crawler(location_no, sub_location_no)
            
            #기존에 pension모델 1차적으로 이름,가격,이미지로  만들던것에
            #location, sub_location 속성 추가해서 이곳에서 만들면 될듯하다.
            # 일단은 리스트 형태로 정보 6개 묶어서 저장해보겠음.
            #[[location,sub_location,title,pricem,img_file,pldx]....팬션 999까지 한 리스트에
            
            for i in range(len(sub_locations_info_list[0])):
                location_info_list.append([location_name,
                                           sub_location_name,
                                           sub_locations_info_list[0][i],# title
                                           sub_locations_info_list[1][i],# price
                                           sub_locations_info_list[2][i],# img_file
                                           sub_locations_info_list[3][i],# pldx
                                           sub_locations_info_list[4][i]
                                          ])
    
    return location_info_list
    
#########################################################################################################






# 가격에 string, 100,00 표현 맞지 않는 경우 0넣고 아니면 int로 바꿔서 출력
def get_int_only(string):
    a= re.findall( '(\d*,\d*)원',string)
    if len(a)==0:
        result= 0
    else:
        result = int(re.sub(',','',a[0]))
    return result 




#############################################################################################
################pension-detail-crawler 작성중.################################################
########
######## 팬션별 세부정보 크롤링.

# p테그 없는것 24140
# 픽업가능타이틀,디테일 없는경우, 이용주의사항 디테일만 없는경우 32928
# 픽업가능타이틀,디테일 없는경우, 27114
# 픽업가능 있는경우 20405


def pension_detail_crawler(ypidx):
    params = {
        'ypIdx':  ypidx
    }

    url = "http://www.yapen.co.kr/details?" + parse.urlencode(params)

    request = requests.get(url)
    response = request.text
    soup = BeautifulSoup(response, 'lxml')

    ##############pension_detail 페이지 상단 기본정보들 .
    # name 
    name_root = soup.select('div.wrap_1000')
    name = name_root[0].select('h3')[0].get_text()                    # name 

    # address 
    table = soup.select('table.pensionTbl')
    trs = table[0].select('tr')
    tds = trs[0].select('td')
    address = tds[0].get_text()                                        # address 

    # check_in, check_out 
    tds2 = trs[1].select('td')
    check_in_out = tds2[0].select('span')
    check_in = check_in_out[0].get_text()                              # check_in
    check_out = check_in_out[1].get_text()                             # check_out 

    # pickup
    tds3 = trs[2].select('td')
    pickup = tds3[0].get_text(strip=True)                              # pickup

    # room_num
    td4 = trs[3].select('td')
    number_tags = td4[0].select('span')
    room_string = number_tags[0].get_text()
    room_num = int(re.search('(\d*)',room_string).group())             # room_num

    # info
    td5 = trs[4].select('td')
    infos = td5[0].select('p')
    info = ''
    for one_info in infos:
        info = info + '\n' + one_info.get_text() + '\n'                # info

    # theme 
    td6 = trs[5].select('td')
    lis = td6[0].select('li')
    theme_list = []
    for li in lis:
        theme_list.append(li.get_text() )
    theme = json.dumps(theme_list)                                     # theme 


    ###############pension_detail 페이지 하단 추가정보들 . 
    detailDiv = soup.select('div.detailDiv')[0]
    detailsPensionInfoTitle = detailDiv.select('div.detailsPensionInfoTitle')

    pension_detail_bellow_dict = dict()                 
    for one_title in detailsPensionInfoTitle:
        next_detail_div = one_title.findNext('div')
        next_detail=''
        for p in next_detail_div.select('p'):
            next_detail= next_detail + p.get_text() +'\n'
        # p 테그없고 바로 text쓴경우에는 검출안된다.....         
        pension_detail_bellow_dict[one_title.get_text(strip=True)] = next_detail

    check_in_out_detail = ""
    pickup_detail = ""
    gretting = ""
    precautions = "" 

    for key, value in pension_detail_bellow_dict.items():
        if key == '입실 / 퇴실시간':
            check_in_out_detail = value                               # check_in_out_detail
        elif key == '픽업가능':
            pickup_detail = value                                     # pickup_detail
        elif key == '펜션소개 및 인사말':
            gretting = value                                          # gretting
        elif key == '이용 주의사항':
            precautions =  value                                      # precautions

#     print("@@@@PensionObject 속성들")
#     print(name)
#     print(address)
#     print(check_in)
#     print(check_out)
#     print(room_num)
#     print(info)
#     print(theme)

#     print(check_in_out_detail)
#     print(pickup_detail) 
#     print(gretting)
#     print(precautions) 
    
#     print("@@@@PensionObject 속성들 타입")
#     print(type(name))
#     print(type(address))
#     print(type(check_in))
#     print(type(check_out))
#     print(type(room_num))
#     print(type(info))
#     print(type(theme))

#     print(type(check_in_out_detail))
#     print(type(pickup_detail)) 
#     print(type(gretting))
#     print(type(precautions)) 
    
    
    
    

    ##############PensionImage 에 넣을 Pension 사진들 3장뽑겠슴.
    pensionImagesLists = soup.find(id="pensionMainImageSlider")
    image_tags = pensionImagesLists.select(" > div img")

    for index,image_tag in enumerate(image_tags):
        if index%2 == 0:
            image_src = image_tag.get("src") #image_src---------------->PensionImage객체만들때써라 
#             print('@@@@팬션 이미지')
#             print(image_src)
            if index == 4: #3장 뽑는 시점에서 break
                break
    

                
    ############################################################
    # Room 모델 정보, RoomImage 모델 체우기 위한 이미지 셀레늄으로 뽑아보겠슴.

    # 버튼 이름 으로쓸 방 이름부터 뽑아내겠슴. 
    room_name_list = []
    image_table = soup.select('div.roomImageLists')
    image_table_lis = image_table[0].select('li')
    for index,image_table_li in enumerate(image_table_lis):
        if 0<index<room_num+1:
            room_name_list.append(image_table_li.get_text())
    

    # 접속.
    chromedriver_dir = '/home/nasanmaro/Desktop/projects/yapen/test/selenium_crawling_test/chromedriver'
    driver = webdriver.Chrome(chromedriver_dir)
    driver.get(url)
    time.sleep(5)

    # 방갯수만큼의 버튼을 클릭!
    image_table = driver.find_element_by_class_name('roomImageLists')
    for room_name_text in room_name_list:                        
        name = room_name_text                                     # 객실 이름.
        room_name_button = image_table.find_elements_by_xpath('//li[contains(text(), "{0}") and @class="roomLists"]'.format(room_name_text))           
#         room_name_button = image_table.find_elements_by_xpath(f"//*[contains(text(), '{room_name_text}')]")
        room_name_button[0].click()
        time.sleep(3) #버튼 클릭후 충분히 멈춰줘야 사진이 로딩된다.

        # 드라이버 인스턴스로부터 현제 메뉴 연상태의 페이지 소스 받아서 source에 넣는다. 
        source = driver.page_source
        # soup 객체로 만듬.
        soup = BeautifulSoup(source, 'lxml')


        pensionImagesLists = soup.find(id="pensionImagesLists")
        print('@@@@@@@@@test_for print@@@@@@@@@@@@2')
        print(pensionImagesLists)
        jssorts=pensionImagesLists.select_one('div.jssort07')
        jssort = jssorts.select('div')[0]
        image_tags = jssort.select('div.p > img')

        for index,image_tag in enumerate(image_tags):
            image_src = image_tag.get("src")  #image_src--------------------->RoomImage객체만들때써라
#             print('@@룸 이미지')
#             print(image_src)
            if index == 2: #3장 뽑는 시점에서 break
                break
                                                  #이for문안에서RoomImage객체 room마다 총세번 만들면될듯

        # room 의 속성들 뽑는중.            
        layer_table_trs = soup.select('div.layerBox > table > tbody > tr')

        td_list1 = layer_table_trs[0].select('td') # 1행
        structure = td_list1[1].get_text()                       #'객실구조'
        size =td_list1[3].get_text(strip=True)                   # '크기'
        num_result = re.findall('(\d*)명 / (\d*)명',td_list1[5].get_text()) 
        normal_num_poeple = int(num_result[0][0])                # '기준인원'
        max_num_people = int(num_result[0][1])                   # '최대인원'

        td_list2 = layer_table_trs[1].select('td') # 2행
        equipments = td_list2[1].get_text()                     # 구비시설 

        td_list3 = layer_table_trs[2].select('td') # 3행
        info = td_list3[1].get_text()                           # 객실 설명

        td_list4 = layer_table_trs[3].select(' > td') # 4행

        ######기본금액뽑기. 
        # price table 의 content중 좌상단 한개만 선택해서 기본금액으로 이것만 저장하려함.
        price_table_td = td_list4[1].select('table.datePriceTbl > tbody > tr > td')

        #추가할인으로 밑줄 그어진 부분 같은 경우는 이것이 p테그안에 들어가서 따로 구분해줌.
        price_p_tag = price_table_td[1].select_one('p')
        if price_p_tag==None:
            price_p_tag = price_table_td[1]
        # 100,000원 에서 숫자들만뽑아냄.
        price= get_int_only(price_p_tag.get_text(strip=True))    # 가격 

        ###### 추가금액 뽑기
        extra_charge_table_trs = td_list4[3].select('table.datePriceTbl > thead > tr')
        extra_charge_head = extra_charge_table_trs[0].get_text(strip=True)  #추가금액 헤드 
        extra_charge_adult_str =  extra_charge_table_trs[1].select('td')[1].get_text(strip=True)
        extra_charge_child_str =  extra_charge_table_trs[2].select('td')[1].get_text(strip=True)
        extra_charge_baby_str = extra_charge_table_trs[3].select('td')[1].get_text(strip=True)
        # 100,000원 에서 숫자들만뽑아냄.
        extra_charge_adult = get_int_only(extra_charge_adult_str)    # 어른 
        extra_charge_child = get_int_only(extra_charge_child_str)    # 아동 
        extra_charge_baby = get_int_only(extra_charge_baby_str)      # 유아
        
        time.sleep(2)
        
#         print("@@RoomObject 속성들")
#         print(name)
#         print(structure)
#         print(size)
#         print(normal_num_poeple)
#         print(max_num_people)
#         print(equipments)
#         print(info)
#         print(price)
        
#         print(extra_charge_head)
#         print(extra_charge_adult)
#         print(extra_charge_child)
#         print(extra_charge_baby)
        
#         print("@@RoomObject 속성들 타입")
#         print(type(name))
#         print(type(structure))
#         print(type(size))
#         print(type(normal_num_poeple))
#         print(type(max_num_people))
#         print(type(equipments))
#         print(type(info))
#         print(type(price))
        
#         print(type(extra_charge_head))
#         print(type(extra_charge_adult))
#         print(type(extra_charge_child))
#         print(type(extra_charge_baby))
        
        
    driver.close()
    

    

    


#@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@
#@@@@@@@@@@@@@@@@@@@@@실행창 @@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@

# province_name_list = province_name_list_crawler()
# print(province_name_list)

# test = location_name_list_crawler()
# print(test)

# test = pension_crawler('015','1.015001')
# print(test)


# location_info_list = location_crawler()
# print(location_info_list)



# 되던것. 
# p테그 없는것 24140
# 픽업가능타이틀,디테일 없는경우, 이용주의사항 디테일만 없는경우 32928
# 픽업가능타이틀,디테일 없는경우, 27114
# 픽업가능 있는경우 20405

# 클릭 안되는것 26104
pension_detail_crawler(26104)


@@@@@@@@@test_for print@@@@@@@@@@@@2
<div id="pensionImagesLists" style="position: relative; width: 800px; height: 600px; overflow: hidden;">
<div style="position: absolute; top: 0px; left: 0px; width: 800px; height: 600px; transform-origin: 0px 0px 0px; transform: scale(1);"><div class="" style="position: relative; width: 800px; height: 600px; overflow: visible; top: 0px; left: 0px;"><div style="cursor: pointer; position: absolute; left: 0px; top: 0px; width: 800px; height: 600px; overflow: hidden; z-index: 0;" u="slides">
<div style="width: 800px; height: 600px; top: 0px; left: 0px; position: absolute; overflow: hidden; display: none;">
<img src="http://img.yapen.co.kr/pension/room/26104/800x0/3547833684_rZHzk7Ma_1.jpg" style="width:800px; height:600px;" u="image"/>
<img src="http://img.yapen.co.kr/pension/room/26104/800x0/3547833684_rZHzk7Ma_1.jpg" style="width:800px; height:600px;" u="thumb"/>
</div>
<div style="width: 800px; height: 600px; top: 0px; left: 0px; position: absolute; 

@@@@@@@@@test_for print@@@@@@@@@@@@2
<div id="pensionImagesLists" style="position: relative; width: 800px; height: 600px; overflow: hidden;">
<div style="position: absolute; top: 0px; left: 0px; width: 800px; height: 600px; transform-origin: 0px 0px 0px; transform: scale(1);"><div class="" style="position: relative; width: 800px; height: 600px; overflow: visible; top: 0px; left: 0px;"><div style="cursor: pointer; position: absolute; left: 0px; top: 0px; width: 800px; height: 600px; overflow: hidden; z-index: 0;" u="slides">
<div style="width: 800px; height: 600px; top: 0px; left: 0px; position: absolute; overflow: hidden; display: none;">
<img src="http://img.yapen.co.kr/pension/room/26104/800x0/3547833685_XZUtJyQo_1.jpg" style="width:800px; height:600px;" u="image"/>
<img src="http://img.yapen.co.kr/pension/room/26104/800x0/3547833685_XZUtJyQo_1.jpg" style="width:800px; height:600px;" u="thumb"/>
</div>
<div style="width: 800px; height: 600px; top: 0px; left: 0px; position: absolute; 

@@@@@@@@@test_for print@@@@@@@@@@@@2
<div id="pensionImagesLists" style="position: relative; width: 800px; height: 600px; overflow: hidden;">
<div style="position: absolute; top: 0px; left: 0px; width: 800px; height: 600px; transform-origin: 0px 0px 0px; transform: scale(1);"><div class="" style="position: relative; width: 800px; height: 600px; overflow: visible; top: 0px; left: 0px;"><div style="cursor: pointer; position: absolute; left: 0px; top: 0px; width: 800px; height: 600px; overflow: hidden; z-index: 0;" u="slides">
<div style="width: 800px; height: 600px; top: 0px; left: 0px; position: absolute; overflow: hidden; display: none;">
<img src="http://img.yapen.co.kr/pension/room/26104/800x0/3547833685_h8TzS4ki_1.jpg" style="width:800px; height:600px;" u="image"/>
<img src="http://img.yapen.co.kr/pension/room/26104/800x0/3547833685_h8TzS4ki_1.jpg" style="width:800px; height:600px;" u="thumb"/>
</div>
<div style="width: 800px; height: 600px; top: 0px; left: 0px; position: absolute; 

@@@@@@@@@test_for print@@@@@@@@@@@@2
<div id="pensionImagesLists" style="position: relative; width: 800px; height: 600px; overflow: hidden;">
<div style="position: absolute; top: 0px; left: 0px; width: 800px; height: 600px; transform-origin: 0px 0px 0px; transform: scale(1);"><div class="" style="position: relative; width: 800px; height: 600px; overflow: visible; top: 0px; left: 0px;"><div style="cursor: pointer; position: absolute; left: 0px; top: 0px; width: 800px; height: 600px; overflow: hidden; z-index: 0;" u="slides">
<div style="width: 800px; height: 600px; top: 0px; left: 0px; position: absolute; overflow: hidden; display: none;">
<img src="http://img.yapen.co.kr/pension/room/26104/800x0/3547833685_t5CmiOTd_1.jpg" style="width:800px; height:600px;" u="image"/>
<img src="http://img.yapen.co.kr/pension/room/26104/800x0/3547833685_t5CmiOTd_1.jpg" style="width:800px; height:600px;" u="thumb"/>
</div>
<div style="width: 800px; height: 600px; top: 0px; left: 0px; position: absolute; 

@@@@@@@@@test_for print@@@@@@@@@@@@2
<div id="pensionImagesLists" style="position: relative; width: 800px; height: 600px; overflow: hidden;">
<div style="position: absolute; top: 0px; left: 0px; width: 800px; height: 600px; transform-origin: 0px 0px 0px; transform: scale(1);"><div class="" style="position: relative; width: 800px; height: 600px; overflow: visible; top: 0px; left: 0px;"><div style="cursor: pointer; position: absolute; left: 0px; top: 0px; width: 800px; height: 600px; overflow: hidden; z-index: 0;" u="slides">
<div style="width: 800px; height: 600px; top: 0px; left: 0px; position: absolute; overflow: hidden; display: none;">
<img src="http://img.yapen.co.kr/pension/room/26104/800x0/3547833685_B3H60a5P_1.jpg" style="width:800px; height:600px;" u="image"/>
<img src="http://img.yapen.co.kr/pension/room/26104/800x0/3547833685_B3H60a5P_1.jpg" style="width:800px; height:600px;" u="thumb"/>
</div>
<div style="width: 800px; height: 600px; top: 0px; left: 0px; position: absolute; 

...

@@@@@@@@@test_for print@@@@@@@@@@@@2
<div id="pensionImagesLists" style="position: relative; width: 800px; height: 600px; overflow: hidden;">
<div style="position: absolute; top: 0px; left: 0px; width: 800px; height: 600px; transform-origin: 0px 0px 0px; transform: scale(1);"><div class="" style="position: relative; width: 800px; height: 600px; overflow: visible; top: 0px; left: 0px;"><div style="cursor: pointer; position: absolute; left: 0px; top: 0px; width: 800px; height: 600px; overflow: hidden; z-index: 0;" u="slides">
<div style="width: 800px; height: 600px; top: 0px; left: 0px; position: absolute; overflow: hidden; display: none;">
<img src="http://img.yapen.co.kr/pension/room/26104/800x0/3547833684_pjgTPceA_13.jpg" style="width:800px; height:600px;" u="image"/>
<img src="http://img.yapen.co.kr/pension/room/26104/800x0/3547833684_pjgTPceA_13.jpg" style="width:800px; height:600px;" u="thumb"/>
</div>
<div style="width: 800px; height: 600px; top: 0px; left: 0px; position: absolute

@@@@@@@@@test_for print@@@@@@@@@@@@2
<div id="pensionImagesLists" style="position: relative; width: 800px; height: 600px; overflow: hidden;">
<div style="position: absolute; top: 0px; left: 0px; width: 800px; height: 600px; transform-origin: 0px 0px 0px; transform: scale(1);"><div class="" style="position: relative; width: 800px; height: 600px; overflow: visible; top: 0px; left: 0px;"><div style="cursor: pointer; position: absolute; left: 0px; top: 0px; width: 800px; height: 600px; overflow: hidden; z-index: 0;" u="slides">
<div style="width: 800px; height: 600px; top: 0px; left: 0px; position: absolute; overflow: hidden; display: none;">
<img src="http://img.yapen.co.kr/pension/room/26104/800x0/3547833685_GuOWedCh_AP2A3652.jpg" style="width:800px; height:600px;" u="image"/>
<img src="http://img.yapen.co.kr/pension/room/26104/800x0/3547833685_GuOWedCh_AP2A3652.jpg" style="width:800px; height:600px;" u="thumb"/>
</div>
<div style="width: 800px; height: 600px; top: 0px; left: 0px; positi

@@@@@@@@@test_for print@@@@@@@@@@@@2
<div id="pensionImagesLists" style="position: relative; width: 800px; height: 600px; overflow: hidden;">
<div style="position: absolute; top: 0px; left: 0px; width: 800px; height: 600px; transform-origin: 0px 0px 0px; transform: scale(1);"><div class="" style="position: relative; width: 800px; height: 600px; overflow: visible; top: 0px; left: 0px;"><div style="cursor: pointer; position: absolute; left: 0px; top: 0px; width: 800px; height: 600px; overflow: hidden; z-index: 0;" u="slides">
<div style="width: 800px; height: 600px; top: 0px; left: 0px; position: absolute; overflow: hidden; display: none;">
<img src="http://img.yapen.co.kr/pension/room/26104/800x0/3547833685_zvjQBm3O_AP2A3777.jpg" style="width:800px; height:600px;" u="image"/>
<img src="http://img.yapen.co.kr/pension/room/26104/800x0/3547833685_zvjQBm3O_AP2A3777.jpg" style="width:800px; height:600px;" u="thumb"/>
</div>
<div style="width: 800px; height: 600px; top: 0px; left: 0px; positi

@@@@@@@@@test_for print@@@@@@@@@@@@2
<div id="pensionImagesLists" style="position: relative; width: 800px; height: 600px; overflow: hidden;">
<div style="position: absolute; top: 0px; left: 0px; width: 800px; height: 600px; transform-origin: 0px 0px 0px; transform: scale(1);"><div class="" style="position: relative; width: 800px; height: 600px; overflow: visible; top: 0px; left: 0px;"><div style="cursor: pointer; position: absolute; left: 0px; top: 0px; width: 800px; height: 600px; overflow: hidden; z-index: 0;" u="slides">
<div style="width: 800px; height: 600px; top: 0px; left: 0px; position: absolute; overflow: hidden; display: none;">
<img src="http://img.yapen.co.kr/pension/room/26104/800x0/3547833685_vg7oQtIc_AP2A3773.jpg" style="width:800px; height:600px;" u="image"/>
<img src="http://img.yapen.co.kr/pension/room/26104/800x0/3547833685_vg7oQtIc_AP2A3773.jpg" style="width:800px; height:600px;" u="thumb"/>
</div>
<div style="width: 800px; height: 600px; top: 0px; left: 0px; positi

@@@@@@@@@test_for print@@@@@@@@@@@@2
<div id="pensionImagesLists" style="position: relative; width: 800px; height: 600px; overflow: hidden;">
<div style="position: absolute; top: 0px; left: 0px; width: 800px; height: 600px; transform-origin: 0px 0px 0px; transform: scale(1);"><div class="" style="position: relative; width: 800px; height: 600px; overflow: visible; top: 0px; left: 0px;"><div style="cursor: pointer; position: absolute; left: 0px; top: 0px; width: 800px; height: 600px; overflow: hidden; z-index: 0;" u="slides">
<div style="width: 800px; height: 600px; top: 0px; left: 0px; position: absolute; overflow: hidden; display: none;">
<img src="http://img.yapen.co.kr/pension/room/26104/800x0/3547833685_WzTxM15r_AP2A3559.jpg" style="width:800px; height:600px;" u="image"/>
<img src="http://img.yapen.co.kr/pension/room/26104/800x0/3547833685_WzTxM15r_AP2A3559.jpg" style="width:800px; height:600px;" u="thumb"/>
</div>
<div style="width: 800px; height: 600px; top: 0px; left: 0px; positi

In [16]:
params = {
        'ypIdx':  26104
    }

url = "http://www.yapen.co.kr/details?" + parse.urlencode(params)

request = requests.get(url)
response = request.text
soup = BeautifulSoup(response, 'lxml')

result = soup.select('div.roomImageLists')
print(result)

[<div class="roomImageLists">
<ul style="max-height:530px; overflow-y:auto; overflow-x:hidden;">
<li class="roomLists" onclick="pensionImageView('26104','16614','E');" style="width:390px; color:#FFF; background-color:#FF6559;">펜션전경</li>
<li class="roomLists" onclick="pensionImageView('26104','53161','R');" style="width:390px;">비쥬</li>
<li class="roomLists" onclick="pensionImageView('26104','53162','R');" style="width:390px;">슈에뜨</li>
<li class="roomLists" onclick="pensionImageView('26104','53163','R');" style="width:390px;">미뇽(스파)</li>
<li class="roomLists" onclick="pensionImageView('26104','53164','R');" style="width:390px;">루나 (스파)</li>
<li class="roomLists" onclick="pensionImageView('26104','53165','R');" style="width:390px;">샤이오</li>
<li class="roomLists" onclick="pensionImageView('26104','53170','R');" style="width:390px;">파미유(스파)</li>
<li class="roomLists" onclick="pensionImageView('26104','53171','R');" style="width:390px;">바또</li>
<li class="roomLists" onclick="pensionImageView